In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
dftrain = pd.read_csv(train_file_path,sep="\t",header=None)
dftrain.columns = ['label', 'text']
dftrain

dftest = pd.read_csv(test_file_path,sep="\t",header=None)
dftest.columns = ['label', 'text']
dftest

data = pd.concat([dftrain, dftest], ignore_index=True)
data

tags = data["label"]
texts = data["text"]
num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)

mat_data = tok.texts_to_matrix(texts,mode='count')
mat_train = tok.texts_to_matrix(dftrain,mode='count')
mat_test = tok.texts_to_matrix(dftest,mode='count')

max_len = 100
x_train = tok.texts_to_sequences(dftrain["text"])
x_test = tok.texts_to_sequences(dftest["text"])
train = pad_sequences(x_train,maxlen=max_len)
max_len = 100
test = pad_sequences(x_test,maxlen=max_len)

labeltrain = dftrain["label"]
traintags = le.fit_transform(labeltrain)
labeltest = dftest["label"]
testtags = le.fit_transform(labeltest)

max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

model = keras.models.Sequential()
model.add(keras.layers.Embedding(max_features, 128))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', keras.metrics.binary_accuracy])
model.summary()

model.fit(train,traintags,batch_size=32,epochs=10,verbose=1,validation_split=0.3)

_,accuracy,_ =model.evaluate(test,testtags)
accuracy

string = ["What you doing?how are you?"]
inputstr = np.array(tok.texts_to_sequences(string))
pred = model.predict(inputstr)
output = [pred[0][0], 'ham']
output

In [ ]:
# function to predict messages based on model
def predict_message(pred_text):

  inputstr = tok.texts_to_sequences([pred_text])
  inputstr = np.array(inputstr)  # Convert list to numpy array
  pred = model.predict(inputstr)[0][0]

  if pred >= 0.5:
    label = "spam"
  else:
    label = "ham"

  output = [pred, label]

  return output

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
